In [1]:
import pulp as pl

# Define the problem
prob = pl.LpProblem("Developer_and_Manager_Assignment_Separated", pl.LpMinimize)

# Developers and Managers data
developers = {"Android": {"junior": 3, "mid": 3, "senior": 2},
              "iOS": {"junior": 3, "mid": 2, "senior": 2}}
managers = {"Android_manager": 1, "iOS_manager": 1}

# Projects data
projects = {"Telecommunications": {"Android": 2, "iOS": 1, "priority": 10, "deadline": 30, "manager": "Android_manager"},
            "Banks": {"Android": 1, "iOS": 2, "priority": 8, "deadline": 20, "manager": "iOS_manager"},
            "Municipalities": {"Android": 1, "iOS": 1, "priority": 5, "deadline": 40, "manager": "Android_manager"},
            "E-commerce": {"Android": 2, "iOS": 2, "priority": 7, "deadline": 25, "manager": "iOS_manager"}}

# Decision variables for developers
dev_assignments = pl.LpVariable.dicts("DevAssignments",
                                      ((p, t, l) for p in projects for t in developers for l in developers[t]),
                                      lowBound=0, upBound=1, cat='Binary')

# Decision variables for managers
mgr_assignments = pl.LpVariable.dicts("ManagerAssignments",
                                      ((p, m) for p in projects for m in managers),
                                      lowBound=0, upBound=1, cat='Binary')

# Objective function to minimize the weighted sum of project duration considering their priority
prob += pl.lpSum([dev_assignments[p, t, l] * (projects[p]["deadline"] / projects[p]["priority"]) 
                  for p in projects for t in developers for l in developers[t]] +
                 [mgr_assignments[p, m] * (projects[p]["deadline"] / projects[p]["priority"])
                  for p in projects for m in managers])

# Developer assignment constraints
for p in projects:
    for t in developers:
        prob += pl.lpSum([dev_assignments[p, t, l] for l in developers[t]]) == projects[p][t]

# Manager assignment constraints
for p in projects:
    prob += mgr_assignments[p, projects[p]["manager"]] == 1

# Ensure each person works on one project at a time
for t in developers:
    for l in developers[t]:
        prob += pl.lpSum([dev_assignments[p, t, l] for p in projects]) <= 1

for m in managers:
    prob += pl.lpSum([mgr_assignments[p, m] for p in projects]) <= managers[m]

# Solve the problem
prob.solve()

# Output results
for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/OR/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/9f/pv1nlhw528d_5zttzbkb_h5m0000gn/T/166d1a7056114b25baf7bc8b03f79e16-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/9f/pv1nlhw528d_5zttzbkb_h5m0000gn/T/166d1a7056114b25baf7bc8b03f79e16-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 25 COLUMNS
At line 182 RHS
At line 203 BOUNDS
At line 236 ENDATA
Problem MODEL has 20 rows, 32 columns and 60 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.01

DevAssignments_('Banks',_'Android',_'senior') = 1.0
DevAssignments_('Banks',_'iOS',_'junior') = 1.0
DevAssignments_('Banks',_'iOS',_'mid') = 1.0
DevAs